# 3. <a id='intro'>Pandas</a>

This practice notebook is **guided by the original Lecture 2** structure. All exercises use **real financial / economic data** from Peru and the US.

**Rule for students:** do *not* paste solutions. Fill the TODO blocks.

## 3.1. <a id='def'>Definition</a>

Pandas is a Python library for working with tabular data (Series and DataFrames), including importing, cleaning, reshaping, and merging datasets.

In [ ]:
# Use postponed evaluation of type annotations (helps with `str | None` on older Python versions).
from __future__ import annotations

# Path: cross-platform file/folder paths.
from pathlib import Path
# hashlib: create stable hashes for cache filenames.
import hashlib
# re: regular expressions for validating/parsing date strings.
import re
# warnings: control warning messages.
import warnings

# numpy: numeric operations + NaN handling.
import numpy as np
# pandas: tables (Series/DataFrame) + parsing dates + IO (parquet).
import pandas as pd

# Hide warnings in notebook output (keeps cells clean; you can remove this while debugging).
warnings.filterwarnings("ignore")

# Define a local folder to store cached downloads.
CACHE_DIR = Path(".cache")
# Create the cache folder if it doesn't exist.
CACHE_DIR.mkdir(exist_ok=True)

# Map Spanish 3-letter month abbreviations to English ones (needed for parsing BCRP date labels).
_ES_TO_EN_MONTH = {
    "Ene": "Jan", "Feb": "Feb", "Mar": "Mar", "Abr": "Apr", "May": "May", "Jun": "Jun",
    "Jul": "Jul", "Ago": "Aug", "Set": "Sep", "Sep": "Sep", "Oct": "Oct", "Nov": "Nov", "Dic": "Dec"
}

def _hash_key(*parts: str) -> str:
    # Create a SHA-256 hash object.
    h = hashlib.sha256()
    # Update the hash with each part (as UTF-8 bytes), plus a separator.
    for p in parts:
        h.update(str(p).encode("utf-8"))
        h.update(b"|")
    # Return a short hash prefix to use in filenames (still very unlikely to collide).
    return h.hexdigest()[:24]

def _normalize_period(code: str, period: str | None) -> str | None:
    # If no period provided, return None.
    if period is None:
        return None
    # Convert to string and trim spaces.
    period = str(period).strip()
    # Use the last 2 characters of the BCRP code to infer frequency (PD daily, PM monthly, PA annual).
    freq = code[-2:].upper() if len(code) >= 2 else ""

    if freq == "PD":  # daily frequency
        # If user passes "YYYY-M" or "YYYY-MM", convert to "YYYY-MM-01" (first day of month).
        if re.fullmatch(r"\d{4}-\d{1,2}", period):
            y, m = period.split("-")
            return f"{int(y):04d}-{int(m):02d}-01"
        # If user passes just "YYYY", convert to "YYYY-01-01".
        if re.fullmatch(r"\d{4}", period):
            return f"{int(period):04d}-01-01"
        # Otherwise keep the period as-is (e.g., already "YYYY-MM-DD").
        return period

    if freq == "PM":  # monthly frequency
        # If user passes "YYYY-MM-DD", convert to "YYYY-M" (month index).
        m = re.fullmatch(r"(\d{4})-(\d{1,2})-(\d{1,2})", period)
        if m:
            y, mo, _ = m.groups()
            return f"{int(y):04d}-{int(mo)}"
        # If user passes "YYYY-MM", convert to "YYYY-M".
        m = re.fullmatch(r"(\d{4})-(\d{1,2})", period)
        if m:
            y, mo = m.groups()
            return f"{int(y):04d}-{int(mo)}"
        # If user passes "YYYY", default to "YYYY-1" (January).
        if re.fullmatch(r"\d{4}", period):
            return f"{int(period):04d}-1"
        # Otherwise keep the period as-is.
        return period

    if freq == "PA":  # annual frequency
        # Extract the year "YYYY" if present at the start.
        m = re.match(r"(\d{4})", period)
        return m.group(1) if m else period

    # If frequency is unknown, return the original period string.
    return period

def _parse_bcrp_period_name(name: str) -> pd.Timestamp:
    # Convert to string and trim.
    s = str(name).strip()

    # --- Case 1: ISO-like strings: "YYYY", "YYYY-MM", "YYYY-MM-DD" ---
    try:
        # Validate ISO-like patterns with regex.
        if re.fullmatch(r"\d{4}(-\d{1,2}){0,2}", s):
            # Convert to datetime; raise on failure.
            return pd.to_datetime(s, errors="raise")
    except Exception:
        # If it fails, continue to other formats.
        pass

    # --- Case 2: Monthly label like "Mar.2020" (often used by BCRP monthly series) ---
    m = re.fullmatch(r"([A-Za-zÁÉÍÓÚÑñ]{3})\.(\d{4})", s)
    if m:
        # Extract Spanish month abbreviation and year.
        mon_es, y = m.groups()
        # Convert Spanish month to English month abbreviation if possible.
        mon = _ES_TO_EN_MONTH.get(mon_es[:3], mon_es[:3])
        # Parse using the specified format "%b.%Y".
        return pd.to_datetime(f"{mon}.{y}", format="%b.%Y", errors="coerce")

    # --- Case 3: Daily label like "18Nov25" or "02Ene97" (DDMonYY) ---
    m = re.fullmatch(r"(\d{2})([A-Za-zÁÉÍÓÚÑñ]{3})(\d{2})", s)
    if m:
        # Extract day, Spanish month abbreviation, 2-digit year.
        d, mon_es, yy = m.groups()
        # Convert Spanish month to English month abbreviation if possible.
        mon = _ES_TO_EN_MONTH.get(mon_es[:3], mon_es[:3])
        # Convert 2-digit year to 4-digit year (00–69 => 2000–2069, else 1900–1999).
        year = 2000 + int(yy) if int(yy) <= 69 else 1900 + int(yy)
        # Parse using "%d%b%Y" (e.g., "18Nov2025").
        return pd.to_datetime(f"{d}{mon}{year}", format="%d%b%Y", errors="coerce")

    # --- Fallback: let pandas try its best; invalid parses become NaT ---
    return pd.to_datetime(s, errors="coerce")

def bcrp_get(series_codes, start: str | None = None, end: str | None = None, lang: str = "esp") -> pd.DataFrame:
    """
    Fetch BCRPData series (JSON API) into a DataFrame.

    Returns columns: ["date", <code1>, <code2>, ...]
    """
    # Try importing requests (needed for HTTP calls). If missing, return empty DataFrame.
    try:
        import requests
    except Exception:
        return pd.DataFrame()

    # Accept one code or multiple codes.
    if isinstance(series_codes, (list, tuple)):
        # Clean each code string.
        codes_list = [str(c).strip() for c in series_codes]
        # BCRP API allows multiple codes joined by '-'.
        codes = "-".join(codes_list)
        # Use the first code to infer frequency for date normalization.
        freq_code = codes_list[0]
    else:
        # Single code (string).
        codes = str(series_codes).strip()
        # Split anyway so we keep a list for consistent column naming.
        codes_list = codes.split("-")
        # Use the first code to infer frequency.
        freq_code = codes_list[0]

    # Normalize start/end based on frequency (daily/monthly/annual).
    start_n = _normalize_period(freq_code, start)
    end_n = _normalize_period(freq_code, end)

    # Build a deterministic cache key and cache filename.
    key = _hash_key("bcrp", codes, start_n or "", end_n or "", lang)
    cache_path = CACHE_DIR / f"bcrp_{key}.parquet"
    # If cached file exists, load it and return immediately.
    if cache_path.exists():
        return pd.read_parquet(cache_path)

    # Base endpoint for the BCRP series API.
    base_url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api"
    # Start building URL parts.
    parts = [base_url, codes, "json"]
    # Add start/end only if both are provided.
    if start_n and end_n:
        parts += [start_n, end_n]
    # Add language parameter (e.g., "esp").
    if lang:
        parts += [lang]
    # Join into final URL string.
    url = "/".join(parts)

    # Make the HTTP request (30s timeout).
    r = requests.get(url, timeout=30)
    # Raise an exception if HTTP status is not 200.
    r.raise_for_status()
    # Parse JSON response body.
    obj = r.json()

    # Get the list of periods (each period has a label and values).
    periods = obj.get("periods", [])
    rows = []
    # Convert the JSON structure into rows for a DataFrame.
    for p in periods:
        # Period label (date-like string).
        name = p.get("name")
        # Values are ordered to match the requested codes.
        vals = p.get("values", [])
        # If API returns a single string, wrap it into a list for consistency.
        if isinstance(vals, str):
            vals = [vals]
        # Skip malformed entries.
        if name is None or not isinstance(vals, list):
            continue
        # Pad/truncate values to match number of codes.
        vals = (vals + [None] * len(codes_list))[:len(codes_list)]
        # Append row: [date_label, value1, value2, ...]
        rows.append([name] + vals)

    # Create a DataFrame with "date" + one column per code.
    df = pd.DataFrame(rows, columns=["date"] + codes_list)
    # If no rows, return an empty DataFrame with the right columns.
    if df.shape[0] == 0:
        return pd.DataFrame(columns=["date"] + codes_list)

    # Parse the "date" strings into actual timestamps.
    df["date"] = df["date"].apply(_parse_bcrp_period_name)
    # Convert each code column to numeric.
    for c in codes_list:
        # Replace known "no data" markers with NaN.
        df[c] = df[c].replace({"n.d.": np.nan, "nd": np.nan, "N.D.": np.nan})
        # Coerce to numeric (invalid -> NaN).
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Drop rows where date failed to parse; sort by date; reset index.
    df = df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
    # Save to parquet cache for faster reruns.
    df.to_parquet(cache_path)
    # Return the cleaned data.
    return df

def bcrp_get_cached_or_empty(series_codes, start: str, end: str) -> pd.DataFrame:
    # Safe wrapper: if network/API fails, return an empty DataFrame with expected columns.
    try:
        return bcrp_get(series_codes, start=start, end=end)
    except Exception:
        # Ensure we return the correct columns even when failing.
        if isinstance(series_codes, (list, tuple)):
            codes_list = [str(c).strip() for c in series_codes]
        else:
            codes_list = [str(series_codes).strip()]
        return pd.DataFrame(columns=["date"] + codes_list)

def yf_download_close_volume(tickers, start: str, end: str) -> pd.DataFrame:
    """
    Download Close and Volume (real market data) using yfinance.
    Returns a DataFrame with columns: ["date","ticker","close","volume"] in long format.
    If download fails, returns an empty DataFrame with those columns.
    """
    # Try importing yfinance. If missing, return empty DataFrame with expected columns.
    try:
        import yfinance as yf
    except Exception:
        return pd.DataFrame(columns=["date","ticker","close","volume"])

    # Allow passing a single ticker or a list/tuple of tickers.
    cols = tickers if isinstance(tickers, (list, tuple)) else [tickers]
    # Create a deterministic cache key.
    key = _hash_key("yf_long", ",".join(cols), start, end)
    # Cache filename for this request.
    cache_path = CACHE_DIR / f"yf_long_{key}.parquet"
    # If cached file exists, load it.
    if cache_path.exists():
        return pd.read_parquet(cache_path)

    try:
        # Download OHLCV data; auto_adjust=True returns adjusted prices.
        data = yf.download(cols, start=start, end=end, auto_adjust=True, progress=False)
        # If nothing returned, return empty DataFrame with expected columns.
        if data.empty:
            return pd.DataFrame(columns=["date","ticker","close","volume"])
        # If multiple tickers, yfinance returns MultiIndex columns: ("Close", ticker), etc.
        if isinstance(data.columns, pd.MultiIndex):
            close = data["Close"].copy()
            vol = data["Volume"].copy()
        else:
            # Single ticker: rename to keep ticker as column label.
            close = data[["Close"]].rename(columns={"Close": cols[0]})
            vol = data[["Volume"]].rename(columns={"Volume": cols[0]})
        # Name the index so it becomes a column after reset_index().
        close.index.name = "date"
        vol.index.name = "date"
        # Convert wide -> long: columns become rows with a "ticker" column.
        long_close = close.reset_index().melt(id_vars="date", var_name="ticker", value_name="close")
        long_vol = vol.reset_index().melt(id_vars="date", var_name="ticker", value_name="volume")
        # Merge close and volume long tables on (date, ticker).
        out = long_close.merge(long_vol, on=["date","ticker"], how="inner").dropna(subset=["close"])
        # Cache to parquet.
        out.to_parquet(cache_path)
        # Return the final long-format DataFrame.
        return out
    except Exception:
        # If anything fails, return an empty DataFrame with expected columns.
        return pd.DataFrame(columns=["date","ticker","close","volume"])

def safe_head(df: pd.DataFrame, n: int = 5) -> pd.DataFrame:
    # If df is a DataFrame, return df.head(n); otherwise return an empty DataFrame.
    return df.head(n) if isinstance(df, pd.DataFrame) else pd.DataFrame()


## 3.2. <a id='series'>Pandas Series</a>

We will use:
- **BCRPData API**: daily PEN/USD exchange rate (buy/sell)
- **Yahoo Finance** via `yfinance`: close/volume for US tickers

Data sources:
- BCRP API help: https://estadisticas.bcrp.gob.pe/estadisticas/series/ayuda/api
- yfinance: https://ranaroussi.github.io/yfinance/


In [ ]:
START = "2022-01-01"
END = "2025-12-18"

# BCRP: daily USD/PEN buy & sell
fx = bcrp_get_cached_or_empty(["PD04637PD","PD04638PD"], start=START, end=END).rename(
    columns={"PD04637PD":"PEN_USD_buy", "PD04638PD":"PEN_USD_sell"}
)

# Yahoo Finance: long-format table (date, ticker, close, volume)
tickers = ["SPY", "QQQ", "TLT", "GLD", "EEM"]
us_mkt = yf_download_close_volume(tickers, start=START, end=END)

fx.shape, us_mkt.shape, safe_head(fx), safe_head(us_mkt)

((584, 3),
 (4970, 4),
         date  PEN_USD_buy  PEN_USD_sell
 0 2022-02-01     3.871333      3.877667
 1 2022-02-02     3.852000      3.857000
 2 2022-02-03     3.858500      3.860833
 3 2022-02-04     3.863000      3.867833
 4 2022-02-07     3.838500      3.845833,
         date ticker      close    volume
 0 2022-01-03    EEM  44.624966  27572700
 1 2022-01-04    EEM  44.470776  24579500
 2 2022-01-05    EEM  43.745167  46425100
 3 2022-01-06    EEM  43.944710  34288700
 4 2022-01-07    EEM  44.343792  32640900)

### 3.2.1. <a id='3.2.1'>From `lists` to `Series`</a>

**Assignment:** create a Series from a Python list using FX mid-rate.

1. Create `PENUSD_mid = (buy + sell)/2`.
2. Take the **last 15 values** as a Python list.
3. Build a `pd.Series` with those values (index can be 0..14).
4. Name the Series `PENUSD_mid_last15`.


In [ ]:
# TODO (students): fill this block
PENUSD_mid_last15 = pd.Series(dtype=float)

# Optional self-check (runs only if you filled it)
if len(PENUSD_mid_last15) > 0:
    print(PENUSD_mid_last15.name, PENUSD_mid_last15.shape)
    print(PENUSD_mid_last15.head())

### 3.2.2. <a id='3.2.2'>From `NumPy array` to `Series`</a>

**Assignment:** create a Series from a NumPy array using US market close prices.

1. Filter `us_mkt` for ticker `SPY`.
2. Extract the `close` column as a NumPy array.
3. Build a `pd.Series` with:
   - data = the NumPy array
   - index = the corresponding dates
4. Compute `mean`, `min`, `max` using Series methods.


In [ ]:
# TODO (students): fill this block
SPY_close_series = pd.Series(dtype=float)

summary_stats = {"mean": np.nan, "min": np.nan, "max": np.nan}

# Optional self-check
if len(SPY_close_series) > 0:
    print(SPY_close_series.head())
    print(summary_stats)

### 3.2.3. <a id='3.2.3'>From `Dictionary` to `Series`</a>

**Assignment:** build a dict and convert to a Series.

1. Using `us_mkt`, compute the **last available close** for each ticker in `tickers`.
2. Store results in a dict: `{ticker: last_close}`.
3. Convert to a Series and sort descending.


In [ ]:
# TODO (students): fill this block
last_close_by_ticker = {}  # dict: ticker -> last close
last_close_series = pd.Series(last_close_by_ticker, dtype=float).sort_values(ascending=False)

# Optional self-check
if len(last_close_series) > 0:
    print(last_close_series)

### 3.2.4. <a id='3.2.4'>`Series` vs `NumPy`</a>

**Assignment:** show why alignment matters.

1. Create two Series:
   - `fx_mid`: FX mid-rate indexed by date
   - `spy_close`: SPY close indexed by date
2. Create a DataFrame by combining them (pandas aligns on dates).
3. Separately, create two NumPy arrays of the same length by truncating to the same number of rows.
4. Explain in markdown why pandas alignment is safer.


In [ ]:
# TODO (students): fill this block
aligned_df = pd.DataFrame()

fx_np = np.array([])
spy_np = np.array([])

# Optional self-check
if isinstance(aligned_df, pd.DataFrame) and aligned_df.shape[0] > 0:
    print(aligned_df.head())
    print("NumPy shapes:", fx_np.shape, spy_np.shape)

### 3.2.5. <a id='3.2.5'>Indexing</a>

**Assignment:** practice `.loc` and `.iloc`.

1. From `last_close_series`, use `.iloc` to take the top 3 tickers.
2. Use `.loc` to select the value for `SPY`.
3. If `SPY` is not present, explain why (in markdown).


In [ ]:
# TODO (students): fill this block
top3 = pd.Series(dtype=float)
spy_value = np.nan

# Optional self-check
if len(top3) > 0:
    print(top3)
print("SPY value:", spy_value)

SPY value: nan


## 3.3. <a id='3.3'>DataFrame</a>

We now practice DataFrame creation and common methods using the same datasets.

### 3.3.1. <a id='3.3.1'>DataFrame Generation</a>

#### From `lists` and `dict` to `DataFrame`

**Assignment:** create a DataFrame of ticker metadata.

1. Make a list of tickers.
2. Make a list of last closes (same order).
3. Make a dict for an extra column, e.g. `{ticker: 'US'}`.
4. Build a DataFrame with columns: `ticker`, `last_close`, `market`.


In [ ]:
# TODO (students): fill this block
tickers_list = []
last_close_list = []
market_dict = {}

ticker_df = pd.DataFrame()

# Optional self-check
if isinstance(ticker_df, pd.DataFrame) and ticker_df.shape[0] > 0:
    print(ticker_df.head())

#### From `lists` and `NumPy` to `DataFrame`

**Assignment:** build a DataFrame from NumPy arrays.

1. Take the `close` column for `SPY` and `QQQ` from `us_mkt`.
2. Convert each to a NumPy array.
3. Build a DataFrame with 2 columns: `SPY_close`, `QQQ_close`.
4. Add a column with row index (0..n-1) named `t`.


In [ ]:
# TODO (students): fill this block
prices_np_df = pd.DataFrame()

# Optional self-check
if isinstance(prices_np_df, pd.DataFrame) and prices_np_df.shape[0] > 0:
    print(prices_np_df.head())

### 3.3.2. <a id='3.3.2'>Indexing</a>

**Assignment:** `.loc` and `.iloc` on DataFrames.

1. Use `.iloc` to take first 5 rows of `us_mkt`.
2. Use `.loc` with a boolean condition to keep only rows where `ticker == 'SPY'`.
3. Select only the columns `date`, `ticker`, `close`.


In [ ]:
# TODO (students): fill this block
first5 = pd.DataFrame()
only_spy = pd.DataFrame()

# Optional self-check
if isinstance(only_spy, pd.DataFrame) and only_spy.shape[0] > 0:
    print(only_spy.head())

### 3.3.3. <a id='3.3.3'>General Methods</a>

**Assignment:** basic methods: `.shape`, `.columns`, `.info`, `.describe`, `.sort_values`.

1. Show `us_mkt.shape` and `us_mkt.columns`.
2. Use `.describe()` on `close` and `volume`.
3. Sort `us_mkt` by `volume` descending and keep top 10 rows.


In [ ]:
# TODO (students): fill this block
desc = pd.DataFrame()
top10_volume = pd.DataFrame()

# Optional self-check
if isinstance(top10_volume, pd.DataFrame) and top10_volume.shape[0] > 0:
    print(top10_volume.head())

### 3.3.4. <a id='3.3.4'>Importing Data</a>

**Assignment:** `to_csv` + `read_csv` using real data.

1. Save a subset of `us_mkt` to `data/us_mkt_sample.csv` (e.g., 500 rows).
2. Read it back using `pd.read_csv`.
3. Rename columns to snake_case.
4. Check dtypes and missing values.


In [ ]:
from pathlib import Path
Path("data").mkdir(exist_ok=True)

# TODO (students): fill this block
us_sample = pd.DataFrame()
us_from_csv = pd.DataFrame()

### 3.3.5. <a id='3.3.5'>Filtering data</a>

**Assignment:** filtering with conditions.

1. Filter `us_mkt` for rows where `close` is above the 90th percentile **within each ticker**.
2. Filter rows with `volume` missing (if any) and count them.
3. Create a filtered DataFrame for tickers `['SPY','GLD']` only.


In [ ]:
# TODO (students): fill this block
high_close = pd.DataFrame()
missing_volume = pd.DataFrame()
spy_gld = pd.DataFrame()

# Optional self-check
if isinstance(spy_gld, pd.DataFrame) and spy_gld.shape[0] > 0:
    print(spy_gld.head())

### 3.3.6. <a id='3.3.6'>Dealing with nulls</a>

**Assignment:** introduce NaNs and handle them.

1. Copy `us_mkt` to `us_mkt_nan`.
2. Set 1% of the `close` values to NaN (fixed random seed).
3. Create two cleaned versions:
   - dropped NaNs
   - filled NaNs with the ticker-specific median close
4. Compare shapes.


In [ ]:
# TODO (students): fill this block
us_mkt_nan = us_mkt.copy()
us_drop = pd.DataFrame()
us_fill = pd.DataFrame()

# Optional self-check (does not error even if you keep defaults)
print("Original:", us_mkt.shape, "With NaNs:", us_mkt_nan.shape)
print("Drop:", us_drop.shape, "Fill:", us_fill.shape)

Original: (4970, 4) With NaNs: (4970, 4)
Drop: (0, 0) Fill: (0, 0)


### 3.3.7. <a id='3.3.7'>Duplicates</a>

**Assignment:** create duplicates and remove them.

1. Create `dup_df` by stacking the last 5 rows of `us_mkt` twice.
2. Use `.duplicated()` to detect duplicates.
3. Use `.drop_duplicates()` to remove duplicates.
4. Verify row counts before/after.


In [ ]:
# TODO (students): fill this block
dup_df = pd.DataFrame()
dup_mask = pd.Series(dtype=bool)
dedup_df = pd.DataFrame()

# Optional self-check
print("dup_df rows:", getattr(dup_df, "shape", (0,0))[0])
print("dedup_df rows:", getattr(dedup_df, "shape", (0,0))[0])

dup_df rows: 0
dedup_df rows: 0


### 3.3.8. <a id='3.3.8'>Groupby</a>

**Assignment:** groupby + aggregation.

1. Group `us_mkt` by `ticker` and compute:
   - mean close
   - median close
   - max volume
2. Rename the resulting columns clearly.
3. Sort by mean close descending.


In [ ]:
# TODO (students): fill this block
ticker_summary = pd.DataFrame()

# Optional self-check
if isinstance(ticker_summary, pd.DataFrame) and ticker_summary.shape[0] > 0:
    print(ticker_summary)

### 3.3.9. <a id='3.3.9'>Reshape</a>

##### From Wide to Long

**Assignment:** melt a wide table.

1. Create a small wide DataFrame with 1 row containing last closes for each ticker.
2. Melt it to long format with columns: `ticker`, `last_close`.

##### From Long to Wide

**Assignment:** pivot back.

3. Using `us_mkt`, create a pivot table with:
   - index = `date`
   - columns = `ticker`
   - values = `close`
4. Keep only the first 50 dates.


In [ ]:
# TODO (students): fill this block
wide_last = pd.DataFrame()
long_last = pd.DataFrame()

wide_close = pd.DataFrame()

# Optional self-check
if isinstance(long_last, pd.DataFrame) and long_last.shape[0] > 0:
    print(long_last.head())
if isinstance(wide_close, pd.DataFrame) and wide_close.shape[0] > 0:
    print(wide_close.head())

### 3.3.10. <a id='3.3.10'>Merge</a>

**Assignment:** merge Peru macro data (BCRP) with US market data (Yahoo).

1. Fetch BCRP monthly policy rate: `PD12301MD`.
2. Create a monthly table from US market data by extracting `year` and `month` from the `date` column.
   Hint: you can use `pd.to_datetime` **only here**.
3. Compute the monthly average close for SPY.
4. Merge policy rate with monthly SPY average using `merge`.
5. Save to `outputs/lecture2_policy_spy_monthly.csv`.


In [ ]:
policy = bcrp_get_cached_or_empty("PD12301MD", start=START, end=END).rename(columns={"PD12301MD":"policy_rate"})

from pathlib import Path
Path("outputs").mkdir(exist_ok=True)

# TODO (students): fill this block
spy_monthly = pd.DataFrame()
policy_monthly = pd.DataFrame()
merged_monthly = pd.DataFrame()

## Homework (assigned)
Deliver ONE notebook that includes:

1. **Series tasks** completed (3.2.1 to 3.2.5) with short markdown explanations.
2. A **ticker summary table** (3.3.8) and a short interpretation (3–5 bullet points).
3. A **reshape demonstration** (3.3.9): wide → long and long → wide.
4. The **monthly merge** between BCRP policy rate and SPY monthly average (3.3.10), saved to CSV.
5. A section titled **"Data Sources"** explaining what BCRP and Yahoo Finance provide.

Submit with clean outputs, and do not introduce extra topics not covered in the original lecture.

## 3.4. <a id='3.4'>References</a>

- Pandas Series: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html
- Pandas DataFrame: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
- Pandas melt: https://pandas.pydata.org/docs/reference/api/pandas.melt.html
- Pandas pivot_table: https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
- Pandas merge: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
- BCRP API help: https://estadisticas.bcrp.gob.pe/estadisticas/series/ayuda/api
- yfinance: https://ranaroussi.github.io/yfinance/
